In [33]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

In [90]:

train = pd.read_csv("train_cat1.csv")
test = pd.read_csv("test_cat1.csv")

In [91]:
#Remomve logerror
drop_columns = ['logerror']
train_features = []

for c in train.columns:
    if c not in drop_columns:
        train_features.append(c)
         
print(train_features)
        


['parcelid', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet15', 'finishedsquarefeet50', 'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'pooltypeid7', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr', 'unitcnt', 'numberofstories', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock', 'transaction_year', 'transaction_month', 'transaction_day', 'transaction_quarter', 'New_YearsSinceBuild']


In [92]:
###
### Define categorical feature indices
###

cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train[c].unique())
    if num_uniques < cat_unique_thresh \
        and not 'squarefeet' in c \
        and not 'sqft' in c \
        and not 'cnt' in c \
        and not 'nbr' in c \
        and not 'sum' in c \
        and not 'transaction' in c \
        and not 'New_YearsSinceBuild' in c \
        and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

Cat features are: ['airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'pooltypeid7', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'assessmentyear']


In [94]:

X_train = train[train_features]
y_train = train.logerror
print(X_train.shape, y_train.shape)


(134627, 43) (134627,)


In [95]:

X_test = test[train_features]
print(X_test.shape)

(33261, 43)


In [96]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=630, learning_rate=0.03,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train, y_train,
        cat_features=cat_feature_inds)
    y_pred += model.predict(X_test)
y_pred /= num_ensembles


  0%|          | 0/5 [00:00<?, ?it/s]

0: learn: 0.06896013	total: 273ms	remaining: 2m 51s
1: learn: 0.06877028	total: 587ms	remaining: 3m 4s
2: learn: 0.06866	total: 821ms	remaining: 2m 51s
3: learn: 0.06859788	total: 1.08s	remaining: 2m 48s
4: learn: 0.06854837	total: 1.32s	remaining: 2m 44s
5: learn: 0.06853187	total: 1.42s	remaining: 2m 27s
6: learn: 0.06850222	total: 1.65s	remaining: 2m 26s
7: learn: 0.0684749	total: 1.91s	remaining: 2m 28s
8: learn: 0.06843851	total: 2.13s	remaining: 2m 26s
9: learn: 0.06839956	total: 2.39s	remaining: 2m 28s
10: learn: 0.06836235	total: 2.6s	remaining: 2m 26s
11: learn: 0.06833657	total: 2.84s	remaining: 2m 26s
12: learn: 0.06831114	total: 3.06s	remaining: 2m 25s
13: learn: 0.06829848	total: 3.34s	remaining: 2m 26s
14: learn: 0.06828375	total: 3.61s	remaining: 2m 27s
15: learn: 0.06826956	total: 3.86s	remaining: 2m 28s
16: learn: 0.06825933	total: 4.3s	remaining: 2m 34s
17: learn: 0.06824847	total: 4.67s	remaining: 2m 38s
18: learn: 0.06824459	total: 4.79s	remaining: 2m 33s
19: learn:

 20%|██        | 1/5 [02:49<11:17, 169.27s/it]

0: learn: 0.06897554	total: 276ms	remaining: 2m 53s
1: learn: 0.06880578	total: 554ms	remaining: 2m 53s
2: learn: 0.06871543	total: 712ms	remaining: 2m 28s
3: learn: 0.06861439	total: 917ms	remaining: 2m 23s
4: learn: 0.06857273	total: 1.08s	remaining: 2m 15s
5: learn: 0.06852846	total: 1.31s	remaining: 2m 15s
6: learn: 0.06850296	total: 1.55s	remaining: 2m 18s
7: learn: 0.06847815	total: 1.77s	remaining: 2m 17s
8: learn: 0.06844647	total: 1.93s	remaining: 2m 13s
9: learn: 0.06839834	total: 2.14s	remaining: 2m 12s
10: learn: 0.06837151	total: 2.37s	remaining: 2m 13s
11: learn: 0.06835983	total: 2.49s	remaining: 2m 8s
12: learn: 0.06835779	total: 2.58s	remaining: 2m 2s
13: learn: 0.0683416	total: 2.84s	remaining: 2m 4s
14: learn: 0.06831774	total: 3.05s	remaining: 2m 5s
15: learn: 0.06830406	total: 3.31s	remaining: 2m 7s
16: learn: 0.06829624	total: 3.59s	remaining: 2m 9s
17: learn: 0.06829364	total: 3.69s	remaining: 2m 5s
18: learn: 0.06828349	total: 3.9s	remaining: 2m 5s
19: learn: 0.

 40%|████      | 2/5 [05:31<08:21, 167.16s/it]

...

 60%|██████    | 3/5 [08:06<05:26, 163.41s/it]

0: learn: 0.06894721	total: 269ms	remaining: 2m 48s
1: learn: 0.06878146	total: 506ms	remaining: 2m 38s
2: learn: 0.0687035	total: 790ms	remaining: 2m 45s
3: learn: 0.06863566	total: 1.04s	remaining: 2m 42s
4: learn: 0.06857341	total: 1.26s	remaining: 2m 37s
5: learn: 0.06853212	total: 1.49s	remaining: 2m 34s
6: learn: 0.06849464	total: 1.66s	remaining: 2m 27s
7: learn: 0.06846011	total: 1.9s	remaining: 2m 27s
8: learn: 0.06842438	total: 2.17s	remaining: 2m 29s
9: learn: 0.06840216	total: 2.31s	remaining: 2m 23s
10: learn: 0.06837374	total: 2.58s	remaining: 2m 25s
11: learn: 0.06835879	total: 2.82s	remaining: 2m 25s
12: learn: 0.0683236	total: 3.02s	remaining: 2m 23s
13: learn: 0.06831084	total: 3.25s	remaining: 2m 23s
14: learn: 0.06829934	total: 3.47s	remaining: 2m 22s
15: learn: 0.06829034	total: 3.73s	remaining: 2m 23s
16: learn: 0.06827678	total: 3.94s	remaining: 2m 22s
17: learn: 0.0682605	total: 4.18s	remaining: 2m 22s
18: learn: 0.06825113	total: 4.46s	remaining: 2m 23s
19: lea

 80%|████████  | 4/5 [10:49<02:43, 163.41s/it]

0: learn: 0.06895398	total: 354ms	remaining: 3m 42s
1: learn: 0.0687878	total: 695ms	remaining: 3m 38s
2: learn: 0.06867956	total: 951ms	remaining: 3m 18s
3: learn: 0.06858436	total: 1.24s	remaining: 3m 13s
4: learn: 0.06853546	total: 1.55s	remaining: 3m 13s
5: learn: 0.06849371	total: 2.02s	remaining: 3m 29s
6: learn: 0.06845893	total: 2.45s	remaining: 3m 37s
7: learn: 0.06841613	total: 2.75s	remaining: 3m 34s
8: learn: 0.06840596	total: 3s	remaining: 3m 26s
9: learn: 0.06840007	total: 3.07s	remaining: 3m 10s
10: learn: 0.06837718	total: 3.36s	remaining: 3m 9s
11: learn: 0.06835334	total: 3.68s	remaining: 3m 9s
12: learn: 0.06833272	total: 4.04s	remaining: 3m 11s
13: learn: 0.06831945	total: 4.35s	remaining: 3m 11s
14: learn: 0.06830816	total: 4.67s	remaining: 3m 11s
15: learn: 0.06829501	total: 5s	remaining: 3m 11s
16: learn: 0.06827978	total: 5.24s	remaining: 3m 9s
17: learn: 0.06826127	total: 5.54s	remaining: 3m 8s
18: learn: 0.06824748	total: 5.83s	remaining: 3m 7s
19: learn: 0.06

100%|██████████| 5/5 [13:50<00:00, 168.53s/it]


In [97]:
submission = pd.DataFrame()
submission["parcelid"] = test['parcelid'].copy()
submission["logerror"] = test['logerror'].copy()
submission["prediction"] = y_pred
ground_truth = submission["logerror"].values
predictions = submission["prediction"].values
abs_error = abs(ground_truth - predictions)
submission["abs_error"] = abs_error

# Calculate mean absolute error
mae = mean_absolute_error(ground_truth, predictions)
print("Mean Absolute Error: {}".format(mae))

Mean Absolute Error: 0.0677197127556112


In [98]:
filename = 'cat_preds_mod1.csv'
print(filename)
submission.to_csv(filename,index=False)

cat_preds_mod1.csv
